Currently, the TikTok Research API has an issue where, if you have a large number of videos to fetch, you need to make multiple requests to get all the videos and you can only fetch 100 videos for each day. For that reason, we recommend you create a list of dates and make a request for each date. For more information, please check this issue: https://github.com/INCT-DD/tiktok-sdk/issues/27

In [1]:
from datetime import datetime, timedelta


def generate_date_list(start_date: str, end_date: str) -> list[str]:
    """
    Generate a list of dates between two dates, inclusive.

    Args:
        start_date (str): The start date as a string in "YYYYMMDD" format.
        end_date (str): The end date as a string in "YYYYMMDD" format.

    Returns:
        list of str: A list containing all dates from the start date to the end date,
                     inclusive, each formatted as "YYYYMMDD".

    Raises:
        ValueError: If the input date strings are not in the correct format or if
                    the start date occurs after the end date.

    Example:
        >>> generate_date_list("20240811", "20240815")
        ['20240811', '20240812', '20240813', '20240814', '20240815']
    """
    date_format = "%Y%m%d"
    start_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(end_date, date_format)
    delta = end_date - start_date
    return [
        (start_date + timedelta(days=i)).strftime(date_format)
        for i in range(delta.days + 1)
    ]


start = "20240816"
end = "20241003"

dates = generate_date_list(start, end)


We recommend you store your client key and secret in the environment variables and call them here.

In [ ]:
from os import environ
from TikTok.Query import Query
from TikTok.Auth import OAuth2
from TikTok.ValidationModels.OAuth2 import RequestHeadersModel, TokenRequestBodyModel

auth: OAuth2 = await OAuth2.authenticate(
    headers=RequestHeadersModel(),
    body=TokenRequestBodyModel(
        client_key=environ["CLIENT_KEY"],
        client_secret=environ["CLIENT_SECRET"],
    ),
)

query = Query(auth)


The library uses tenacity to handle retries, but you can also implement your own retry logic as seen below.

In [ ]:
from TikTok.ValidationModels.Video import (
    VideoQueryRequestBuilder,
    VideoQueryOperation,
    VideoQueryFieldName,
    VideoQueryFields,
    VideoQueryResponseModel,
)
import asyncio
from tqdm import tqdm


async def fetch_videos(
    username: str, date_list: list[str], max_retries: int = 3
) -> list[VideoQueryResponseModel]:
    """
    Fetches videos for a given username over multiple dates, with a progress bar and retry logic.

    Args:
        username (str): The TikTok username to query.
        date_list (list): A list of date strings in "YYYYMMDD" format where each date
                          serves as both the start and end date for the query.
        max_retries (int): Maximum number of retries for failed API calls.

    Returns:
        list: A list containing all fetched VideoQueryResponseModel instances.
    """
    all_videos = []

    for date in tqdm(date_list, desc="Processing Dates", unit="date"):
        video_query = VideoQueryRequestBuilder()

        initial_request = (
            video_query.start_date(date)
            .end_date(date)
            .max_count(100)
            .and_(VideoQueryOperation.EQ, VideoQueryFieldName.username, [username])
            .build()
        )

        current_request = initial_request.model_copy()
        has_more = True
        retries = 0

        while has_more:
            try:
                video_query_response = await query.video.search(
                    request=current_request,
                    fields=[
                        VideoQueryFields.id,
                        VideoQueryFields.video_description,
                        VideoQueryFields.create_time,
                        VideoQueryFields.region_code,
                        VideoQueryFields.share_count,
                        VideoQueryFields.view_count,
                        VideoQueryFields.like_count,
                        VideoQueryFields.comment_count,
                        VideoQueryFields.music_id,
                        VideoQueryFields.hashtag_names,
                        VideoQueryFields.username,
                        VideoQueryFields.effect_ids,
                        VideoQueryFields.playlist_id,
                        VideoQueryFields.favorites_count,
                    ],
                )
            except Exception as e:
                retries += 1
                if retries > max_retries:
                    break
                await asyncio.sleep(2**retries)  # Exponential backoff
                continue

            retries = 0

            if hasattr(video_query_response.data, "videos"):
                all_videos.extend(video_query_response.data.videos)

            has_more = getattr(video_query_response.data, "has_more", False)

            if has_more:
                cursor = getattr(video_query_response.data, "cursor", "")
                search_id = getattr(video_query_response.data, "search_id", "")
                if not cursor or not search_id:
                    break

                video_query = VideoQueryRequestBuilder()
                current_request = (
                    video_query.start_date(date)
                    .end_date(date)
                    .max_count(100)
                    .and_(
                        VideoQueryOperation.EQ, VideoQueryFieldName.username, [username]
                    )
                    .cursor(cursor)
                    .search_id(search_id)
                    .build()
                )

    return all_videos


If you want to fetch videos from multiple users, you can do it as follows:

In [ ]:
async def fetch_videos_for_usernames(
    usernames: list[str], date_list: list[str], max_retries: int = 3
) -> list[VideoQueryResponseModel]:
    """
    Fetches videos for multiple usernames over multiple dates, with nested progress bars.

    Args:
        usernames (list[str]): A list of TikTok usernames to query.
        date_list (list[str]): A list of date strings in "YYYYMMDD" format where each date
                          serves as both the start and end date for the query.
        max_retries (int): Maximum number of retries for failed API calls.

    Returns:
        list: A list containing all fetched VideoQueryResponseModel instances from all usernames.
    """
    all_videos = []

    for username in tqdm(usernames, desc="Processing Usernames", unit="username"):
        print(f"Processing: {username}")
        user_videos = await fetch_videos(username, date_list, max_retries)
        all_videos.extend(user_videos)

    return all_videos


In [ ]:
# Remember to replace the list of profiles with your own list of usernames.
videos = await fetch_videos_for_usernames(profiles, dates)


Once you have the list of videos, you can dump it to a dictionary. If you want to load it to a database, you can use the following code:

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(
    [video.model_dump(by_alias=True, exclude_none=True) for video in videos]
)

df.fillna(0, inplace=True)  # Replace NaN with 0. Optional.
